<a href="https://colab.research.google.com/github/antalvdb/mblm/blob/main/timbl_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Memory-based language modeling

Looking for an LLM that is relatively eco-friendly? MBLMs rely on CPUs; no GPUs or TPUs are required. Training MBLMs is costly in terms of RAM, but not in terms of time or computing resources. Running an MBLM in autoregressive GPT-style mode also costs RAM, but still relies on CPUs and is reasonably fast as well, depending on the selected approximation of k-nearest neighbor classification.

In [ ]:
from transformers import AutoTokenizer, AutoModel
import re
import time
import argparse
import sys
import ast

Installing `python3-timbl`, Python bindings for the TiMBL engine.

In [ ]:
!pip install python3-timbl

import timbl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 2.5 MB/s eta 0:00:00


Downloading an example MBLM model (approx. 457MB) with git lfs. This may take a while.

In [ ]:
!git clone https://github.com/antalvdb/mblm
%cd mblm
!git lfs pull -I edufineweb_train_000001-1k_tok.l16r0.ibase
#!git lfs pull -I chatbot-instruction-prompts-100k_tok.l16r0.ibase
#!git lfs pull -I chatbot-instruction-prompts_tok.l16r0.igtree.ibase
%cd

Cloning into 'mblm'...
remote: Enumerating objects: 215, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 215 (delta 36), reused 17 (delta 9), pack-reused 147 (from 1)
Receiving objects: 100% (215/215), 1.19 MiB | 4.47 MiB/s, done.
Resolving deltas: 100% (115/115), done.
Filtering content: 100% (7/7), 5.09 GiB | 36.95 MiB/s, done.
/content/mblm
/root


In [ ]:
# Global verbosity level
VERBOSITY = 3

In [ ]:
def log(message, level=1):
    """Logs a message if the verbosity level is sufficient."""
    if VERBOSITY >= level:
        print(message)

Setting functions for padding the prompt with zeroes, and for generating text

In [ ]:
def pad_prompt(words, max_len=16):
    """Pad or trim the list of words to make it exactly `max_len` words."""
    if words is None:
        words = []  # Ensure words is a list
    if len(words) < max_len:
        words = ['_'] * (max_len - len(words)) + words
    else:
        words = words[-max_len:]
    return words

In [ ]:
def generate_text_from_api(classifier, initial_prompt, max_words=200):
    # Tokenize the initial prompt and convert tokens back to words
    initial_tokens = tokenizer.tokenize(initial_prompt)

    if initial_tokens is None:
        log("Tokenization failed; 'initial_tokens' is None.", level=1)
        initial_tokens = []

    # Prepare the initial prompt, padded or trimmed to 16 words
    padded_instances = []

    # Generate padded instances for next-token predictions
    for i in range(len(initial_tokens)):
        # Take the tokens up to the current position and pad them
        instance = pad_prompt(initial_tokens[:i], max_len=16)
        padded_instances.append((instance, initial_tokens[i] if i < len(initial_tokens) else '_'))

    # Add instances to memory
    for input_instance, next_token in padded_instances:
        log(f"memorized: {input_instance} {next_token}", level=2)
        classifier.append(input_instance, next_token)

    # Use the final part of the prompt for further generation
    prompt_words = pad_prompt(initial_tokens)

    generated_tokens = prompt_words[:]  # Store the full generated text

    try:
        # Loop until max words generated or a period token is found
        for _ in range(max_words):
            next_word = None

            classlabel, distribution, distance = classifier.classify(prompt_words)

            neighbors_output = classifier.bestNeighbours()

            log(f"Neighbors output: {neighbors_output}", level=3)


            # Add instance to instance base
            classifier.append(prompt_words, classlabel)

            log(f"Prompt words: {prompt_words}", level=2)
            log(f"Classlabel: {classlabel}", level=2)
            log(f"Distribution: {distribution}", level=3)
            log(f"Distance: {distance}", level=3)

            generated_tokens.append(classlabel)

            # Shift prompt words and add the new word
            prompt_words = prompt_words[1:] + [classlabel]

            # Stop if a period is generated
            # if classlabel == ".":
            #     break

        # Detokenize the generated tokens
        generated_text = tokenizer.convert_tokens_to_string(generated_tokens)

        # Strip off original padding characters
        generated_text = generated_text.replace("_", "").strip()

        # Print the final generated text
        log(f"Generated text: {generated_text}", level=1)

    except Exception as e:
        log(f"Error: {e}", level=1)


In [ ]:
if __name__ == "__main__":

    # Simulate command-line arguments for notebook environment
    # sys.argv = ['script_name', '--classifier', '/content/mblm/chatbot-instruction-prompts_tok.l16r0.igtree', '--tokenizer', 'bert-base-cased', '--timbl_args', '-a1 +D', '--verbosity', '3']
    sys.argv = ['script_name', '--classifier', '/content/mblm/edufineweb_train_000001-1k_tok.l16r0', '--tokenizer', 'bert-base-cased', '--timbl_args', '-a0 -k9 +v n+db+k+di', '--verbosity', '3']

    # Parse command-line arguments
    parser = argparse.ArgumentParser(description="Memory-based text generator")
    parser.add_argument("--classifier", type=str, required=True, help="Path to the Timbl classifier file")
    parser.add_argument("--tokenizer", type=str, required=True, help="Name of the Hugging Face tokenizer")
    parser.add_argument("--timbl_args", type=str, required=True, help="Timbl arguments as a single string (e.g., '-a4 +D')")
    parser.add_argument("--verbosity", type=int, default=0, help="Verbosity level (0: silent, 1: basic, 2: detailed, 3: debug)")
    args = parser.parse_args()

    # Set global verbosity level
    VERBOSITY = args.verbosity

    # Initialize the tokenizer
    tokenizer = AutoTokenizer.from_pretrained(args.tokenizer)

    # Initialize the classifier
    log("Loading TiMBL Classifier...", level=1)
    classifier = timbl.TimblClassifier(args.classifier, args.timbl_args)
    classifier.load()

    # Loop to continuously ask for input and classify
    while True:
        # Take input from the user
        user_input = input("Please enter prompt (or type 'exit' to quit): ")

        # Check if the user wants to exit
        if user_input.lower() == 'exit':
            log("Exiting.", level=1)
            break

        # Pass the input to the classifier function
        generate_text_from_api(classifier, user_input)


Calling Timbl API : -F Tabbed -a0 -k9 +v n+db+k+di


Streaming output truncated to the last 5000 lines.
#	_	_	_	_	_	_	_	_	_	_	_	_	_	heard	,	50	{ million 1.00000 }
#	_	_	_	_	_	_	_	_	_	_	_	_	heard	,	50	million	{ tons 1.00000 }
#	_	_	_	_	_	_	_	_	_	_	_	heard	,	50	million	tons	{ of 1.00000 }
#	_	_	_	_	_	_	_	_	_	_	heard	,	50	million	tons	of	{ soil 1.00000 }
#	_	_	_	_	_	_	_	_	_	heard	,	50	million	tons	of	soil	{ were 1.00000 }
# k=3, 1 Neighbor(s) at distance: 	4.3839641266579
#	_	_	_	_	_	_	_	_	heard	,	50	million	tons	of	soil	were	{ blown 1.00000 }
# k=4, 1 Neighbor(s) at distance: 	4.9153638019965
#	_	_	_	_	_	_	_	heard	,	50	million	tons	of	soil	were	blown	{ away 1.00000 }
# k=5, 1 Neighbor(s) at distance: 	5.3803406837151
#	_	_	_	_	_	heard	,	50	million	tons	of	soil	were	blown	away	,	{ ” 1.00000 }
# k=6, 1 Neighbor(s) at distance: 	5.4469968285591
#	_	_	_	_	_	_	heard	,	50	million	tons	of	soil	were	blown	away	{ , 1.00000 }
# k=7, 1 Neighbor(s) at distance: 	6.5083712785563
#	_	_	_	_	heard	,	50	million	tons	of	soil	were	blown	away	,	”	{ American 1